In [89]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)  

In [ ]:
df = pd.read_csv(r"Traffic_Violations.csv")

## Create Unique Index

In [ ]:
df["Stop ID"] = df["SeqID"] + df['Time Of Stop']

del df['SeqID'], df['Time Of Stop']

In [ ]:
def assign_indices(dataframe, column):
    
    unique_set = set(dataframe[column])
    
    unique_dict = {}
    
    label = 0
    
    for indices in unique_set:
        unique_dict[indices] = label
        label += 1
        
    return [unique_dict[x] for x in dataframe[column]]

In [ ]:
df['Unique Index'] = assign_indices(df, 'Stop ID')

del df['Stop ID']

In [ ]:
df = df.set_index('Unique Index')
df = df.sort_index()

In [ ]:
"""
Convert df['Geolocation'] from STR --> TUP
"""

df['Geolocation'] = [tuple(x.replace(')','').replace('(','').split(',')) for x in df['Geolocation']]

In [ ]:
df['Search Conducted'] = df['Search Conducted'].fillna("No")

## Aggregate All Results of A Stop

In [ ]:
# """
# Completely Redundant - Dataset already Grouped
# """

# def aggregate_per_stop(dataframe, column, trait):
    
#     index_dict = {}
    
#     for index in dataframe.index:
#         index_dict[index] = 0
# #         if trait in list([dataframe.loc[index][column]]):
#         try:
#             if any(x==trait for x in dataframe.loc[index][column].tolist()):
#                 index_dict[index] = 1        
#                 pass
#         except:
#             if dataframe.loc[index][column] == trait:
#                 index_dict[index] = 1        
#                 pass
    
#     return [index_dict[x] == 1 for x in dataframe.index]
#     #now map dict to list and make new column
            

In [ ]:
# df['Search Conducted - Grouped'] = aggregate_per_stop(df, 'Search Conducted', 'Yes')
# # df['']

In [ ]:
# def test(n):
    
#     ttt = set()
    
#     for i, x in enumerate(df['Search Conducted']):
#         if x == "Yes":
#             ttt.add(df.index[i])
#             n -= 1
#             if n==0:
#                 return(ttt)
            
# test(5)

## Test to see if all citations of a stop have the same characteristics

(e.g. for an arrest with 6 citations, each of the 6 citation-entries has "True" for Arrest

In [ ]:
'''test for consistency across multiple entries for the same stop'''

#create function to test
def test_consistency(ind):
    test_dict = {}
    for col in df.loc[ind].columns:
        test_dict[col] = set([x for x in df.loc[ind][col].tolist()])
        
    for col in df:
#         if len(set([x for x in df.loc[ind][test_column].tolist()])) > 1:
        if len(test_dict[col]) > 1 and col not in ('Description', 'Charge'): #for testing all columns
            print(col)
            print(test_dict[col])
            print(df.loc[ind][col])
            
            
#run tests on repeated indices for all columns
for ind in df[df.index.duplicated(keep=False)].index.unique():
    test_consistency(ind)

In [ ]:
'''test for consistency across multiple entries for the same stop'''

#define columns to test for consistency
TEST_COLS = ['Search Conducted', 'Search Reason']

#create function to test
def test_specific_consistency(ind, TEST_COLS):
    test_dict = {}
    for col in TEST_COLS:
        test_dict[col] = set([x for x in df.loc[ind][col].tolist()])
        
    for col in TEST_COLS:
        if len(set([x for x in df.loc[ind][col].tolist()])) > 1 and not all(y.isnan() for y in df.loc[ind][col].tolist() if type(y)=='numpy.ufunc'):
            print(col)
            print(test_dict[col])
            print(df.loc[ind][col])
            
            
#run tests on repeated indices for all columns


for ind in df[df.index.duplicated(keep=False)].index.unique():
    test_specific_consistency(ind, TEST_COLS)

This doesn't return anything, so we know that the columns in TEST_COLS are consistent

## Export Relevant Columns

In [82]:
df2 = df.head(10).loc[~df.head(10).index.duplicated(keep='first')]

In [90]:
df2

,Date Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,Belts,Personal Injury,Property Damage,Fatal,Commercial License,HAZMAT,Commercial Vehicle,Alcohol,Work Zone,Search Conducted,Search Disposition,Search Outcome,Search Reason,Search Reason For Stop,Search Type,Search Arrest Reason,State,VehicleType,Year,Make,Model,Color,Violation Type,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
Unique Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,10/27/2016,MCP,"3rd District, Silver Spring",DRIVER FAILURE TO OBEY PROPERLY PLACED TRAFFIC CONTROL DEVICE INSTRUCTIONS,2ND @ SPRING ST,38.998705,-77.035473,No,No,No,No,No,No,No,No,No,No,No,NaN,Warning,NaN,21-201(a1),NaN,NaN,MD,02 - Automobile,2015.0,VOLK,4D,"BLUE, LIGHT",Warning,21-201(a1),Transportation Article,False,WHITE,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.998705, -77.0354733333333)"
1,07/28/2014,MCP,"6th District, Gaithersburg / Montgomery Village",HEADLIGHTS (*),S. SCHOOL @ WELBECK,39.188012,-77.185453,No,No,No,No,No,No,No,No,No,No,No,NaN,SERO,NaN,55*,NaN,NaN,MD,02 - Automobile,2000.0,MITS,4S,TAN,ESERO,55*,NaN,False,WHITE,M,MONTGOMERY VILLAGE,MD,MD,A - Marked Patrol,"(39.1880116666667, -77.1854533333333)"
2,03/18/2015,MCP,"6th District, Gaithersburg / Montgomery Village",OPERATING VEHICLE ON HIGHWAY WITH UNAUTHORIZED WINDOW TINTING MATERIAL,355 N/B & PEONY DR,39.139640,-77.196097,No,No,No,No,No,No,No,No,No,No,No,NaN,Warning,NaN,22-406(i1),NaN,NaN,MD,05 - Light Duty Truck,1999.0,CHEV,TK,"BLUE, DARK",Warning,22-406(i1),Transportation Article,False,WHITE,M,ROCKVILLE,MD,MD,A - Marked Patrol,"(39.13964, -77.1960966666667)"
3,12/18/2014,MCP,"3rd District, Silver Spring",EXCEEDING THE POSTED SPEED LIMIT OF 35 MPH,GEORGIA AVE @ BALLART ST,38.997763,-77.030053,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,VA,02 - Automobile,2007.0,TOYOTA,CAMRY,WHITE,Warning,21-801.1,Transportation Article,False,BLACK,F,SILVER SPRING,MD,VA,Q - Marked Laser,"(38.9977633333333, -77.0300533333333)"
4,12/15/2018,MCP,"6th District, Gaithersburg / Montgomery Village","DRIVING WHILE LIC. SUSP. UNDER 16-203, 16-206A2 FAIL TO ATTEND DIP, 17-106, 26-204/206, 27-103",FIELDS RD @ RIO,39.115953,-77.202375,No,No,No,No,No,No,No,No,No,No,No,NaN,Citation,NaN,21-801(a),NaN,NaN,MD,02 - Automobile,2017.0,JAGUAR,SUV,BLACK,Citation,16-303(h),Transportation Article,False,BLACK,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.1159533333333, -77.202375)"


In [88]:
df.head(10)

,Date Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,Belts,Personal Injury,Property Damage,Fatal,Commercial License,HAZMAT,Commercial Vehicle,Alcohol,Work Zone,Search Conducted,Search Disposition,Search Outcome,Search Reason,Search Reason For Stop,Search Type,Search Arrest Reason,State,VehicleType,Year,Make,Model,Color,Violation Type,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
Unique Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,10/27/2016,MCP,"3rd District, Silver Spring",DRIVER FAILURE TO OBEY PROPERLY PLACED TRAFFIC CONTROL DEVICE INSTRUCTIONS,2ND @ SPRING ST,38.998705,-77.035473,No,No,No,No,No,No,No,No,No,No,No,NaN,Warning,NaN,21-201(a1),NaN,NaN,MD,02 - Automobile,2015.0,VOLK,4D,"BLUE, LIGHT",Warning,21-201(a1),Transportation Article,False,WHITE,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.998705, -77.0354733333333)"
1,07/28/2014,MCP,"6th District, Gaithersburg / Montgomery Village",HEADLIGHTS (*),S. SCHOOL @ WELBECK,39.188012,-77.185453,No,No,No,No,No,No,No,No,No,No,No,NaN,SERO,NaN,55*,NaN,NaN,MD,02 - Automobile,2000.0,MITS,4S,TAN,ESERO,55*,NaN,False,WHITE,M,MONTGOMERY VILLAGE,MD,MD,A - Marked Patrol,"(39.1880116666667, -77.1854533333333)"
2,03/18/2015,MCP,"6th District, Gaithersburg / Montgomery Village",OPERATING VEHICLE ON HIGHWAY WITH UNAUTHORIZED WINDOW TINTING MATERIAL,355 N/B & PEONY DR,39.139640,-77.196097,No,No,No,No,No,No,No,No,No,No,No,NaN,Warning,NaN,22-406(i1),NaN,NaN,MD,05 - Light Duty Truck,1999.0,CHEV,TK,"BLUE, DARK",Warning,22-406(i1),Transportation Article,False,WHITE,M,ROCKVILLE,MD,MD,A - Marked Patrol,"(39.13964, -77.1960966666667)"
3,12/18/2014,MCP,"3rd District, Silver Spring",EXCEEDING THE POSTED SPEED LIMIT OF 35 MPH,GEORGIA AVE @ BALLART ST,38.997763,-77.030053,No,No,No,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,VA,02 - Automobile,2007.0,TOYOTA,CAMRY,WHITE,Warning,21-801.1,Transportation Article,False,BLACK,F,SILVER SPRING,MD,VA,Q - Marked Laser,"(38.9977633333333, -77.0300533333333)"
4,12/15/2018,MCP,"6th District, Gaithersburg / Montgomery Village","DRIVING WHILE LIC. SUSP. UNDER 16-203, 16-206A2 FAIL TO ATTEND DIP, 17-106, 26-204/206, 27-103",FIELDS RD @ RIO,39.115953,-77.202375,No,No,No,No,No,No,No,No,No,No,No,NaN,Citation,NaN,21-801(a),NaN,NaN,MD,02 - Automobile,2017.0,JAGUAR,SUV,BLACK,Citation,16-303(h),Transportation Article,False,BLACK,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.1159533333333, -77.202375)"
4,12/15/2018,MCP,"6th District, Gaithersburg / Montgomery Village",PERSON DRIVING MOTOR VEHICLE ON HIGHWAY OR PUBLIC USE PROPERTY ON SUSPENDED LICENSE AND PRIVILEGE,FIELDS RD @ RIO,39.115953,-77.202375,No,No,No,No,No,No,No,No,No,No,No,NaN,Citation,NaN,21-801(a),NaN,NaN,MD,02 - Automobile,2017.0,JAGUAR,SUV,BLACK,Citation,16-303(c),Transportation Article,False,BLACK,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.1159533333333, -77.202375)"
4,12/15/2018,MCP,"6th District, Gaithersburg / Montgomery Village",DRIVING MOTOR VEHICLE ON HIGHWAY WITHOUT REQUIRED LICENSE AND AUTHORIZATION,FIELDS RD @ RIO,39.115953,-77.202375,No,No,No,No,No,No,No,No,No,No,No,NaN,Citation,NaN,21-801(a),NaN,NaN,MD,02 - Automobile,2017.0,JAGUAR,SUV,BLACK,Citation,16-101(a1),Transportation Article,False,BLACK,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.1159533333333, -77.202375)"
4,12/15/2018,MCP,"6th District, Gaithersburg / Montgomery Village",WINDOW TINT,FIELDS RD @ RIO,39.115953,-77.202375,No,No,No,No,No,No,No,No,No,No,No,NaN,Citation,NaN,21-801(a),NaN,NaN,MD,02 - Automobile,2017.0,JAGUAR,SUV,BLACK,ESERO,61,NaN,False,BLACK,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.1159533333333, -77.202375)"
4,12/15/2018,MCP,"6th District, Gaithersburg / Montgomery Village",FAILURE OF INDIVIDUAL DRIVING ON HIGHWAY TO DISPLAY LICENSE TO UNIFORMED POLICE ON DEMAND,FIELDS RD @ RIO,39.115953,-77.202375,No,No,No,No,No,No,No,No,No,No,No,NaN,Citation,NaN,21-801(a),NaN,NaN,MD,02 - Automobile,2017.0,JAGUAR,SUV,BLACK,Citation,16-112(c),Transport

export (weighted by entries) for citations \
export (unique) for everything else

### Export Weighted by Entries (Easy)

arrest, search conducted, probable cause, accident